In [1]:
#Working with files and reading RSS XML's
import os
import feedparser

#These are very useful for data analysis!
import numpy as np
import pandas as pd

#NER Libraries
import natasha

#This is for working with raw Russian texts
import pymorphy2
import nltk
import re

#Networks
import transliterate
import networkx as nx
import gc

In [2]:
#Generator function that gives out articles one by one
def articleGenerator(folderPath):
    #Get names of all files in the target folder
    files = os.listdir(folderPath)
    for file in files:
        
        #If the current file is not an XML, we skip it
        #and go to the next one
        if not file.endswith(".xml"):
            continue
        
        #Create a full path to the file
        fullPath = folderPath + "/" + file
        print("Reading file", fullPath)
        
        #Parse the file using the feedparser library
        feed = feedparser.parse(fullPath)
        
        #If feed.bozo is not 0, this means that persing has failed
        #somehow. However, we technically can continue parsing,
        #so we just alert the user and go on
        if feed.bozo != 0:
            print("FAILED TO PARSE XML CORRECTLY!")
            print(feed.bozo_exception)
            print(feed.bozo_exception.getMessage())
        
        #For each entry in the feed, we yield the full text
        #contained in .summary
        for item in feed.entries:
            yield item.summary

In [4]:
texts = [text for text in articleGenerator(".")]
texts[1]

Reading file ./20_02_2019_13_36_08.xml


'Удальцов решил укрепить движение перед акциями протеста и выборами\nЛидер «Левого фронта» (ЛФ) Сергей Удальцов объявил сбор пожертвований на развитие региональных отделений движения. Средства также нужны для модернизация YouTube-канала и проведения протестных акций. И, конечно, для участия в выборах, на которые, как выяснила «НГ», у Удальцова есть виды. Преобразовывать же движение в партию он пока не планирует, в нынешнем виде удобнее вступать в коалиции и союзы и с левыми, и с правыми политструктурами.\nКак сообщил «НГ» Удальцов, сейчас идет большая работа по организации протестных акций против антисоциальной политики властей. Ближайшая будет 23 февраля, а 17 марта пройдут протесты в защиту Курил и честных выборов. Здесь ЛФ будет действовать вместе с КПРФ. \n«Мы активно готовимся и к выборам - в муниципальные и региональные собрания, на которые пойдем практически во всех регионах. Кандидатов в губернаторы выдвинем совместно с КПРФ - мы вообще выступаем за проведение «левых праймериз»

In [5]:
#Do NER on the text from above, show matches and diagnostics
nameExtr = natasha.NamesExtractor()
matches1 = nameExtr(texts[1])
for match in matches1:
    print(match.span, match.fact)

[0, 8) Name(first=None, middle=None, last='удальцов', nick=None)
[94, 109) Name(first='сергей', middle=None, last='удальцов', nick=None)
[334, 343) Name(first=None, middle=None, last='удальцов', nick=None)
[525, 533) Name(first=None, middle=None, last='удальцов', nick=None)
[1205, 1213) Name(first=None, middle=None, last='удальцов', nick=None)
[1615, 1623) Name(first=None, middle=None, last='удальцов', nick=None)
[2068, 2077) Name(first=None, middle=None, last='удальцов', nick=None)
[2343, 2362) Name(first='анастасий', middle=None, last='удальцов', nick=None)
[2439, 2447) Name(first=None, middle=None, last='удальцов', nick=None)
[2641, 2649) Name(first=None, middle=None, last='удальцов', nick=None)
[2706, 2724) Name(first='алексей', middle=None, last='навальный', nick=None)
[3073, 3081) Name(first=None, middle=None, last='удальцов', nick=None)
[3138, 3156) Name(first='владимир', middle=None, last='квачков', nick=None)
[3203, 3219) Name(first='анатолий', middle=None, last='чубайс', nick

In [6]:
#For now, we only keep the last names
def nameNormalizer(match):
    return match.fact.last


nameNormalizer(matches1[0])

'удальцов'

In [7]:
#Now, we can transform a list of matches into a set of names
def makeNameSet(matches):
    names = set()
    for m in matches:
        name = nameNormalizer(m)
        if name is None:
            continue
        names.add(name)
    return names

makeNameSet(matches1)

{'квачков', 'колядин', 'навальный', 'удальцов', 'чубайс'}

In [9]:
#We can use this to build a network
ex_network = nx.Graph()
n = 0
for text in texts:
    if n % 10 == 0:
        print(n)
    n += 1
    matches = nameExtr(text)
    names = makeNameSet(matches)
    names = list(names) #We want an ordered collection here
    for i in range(0, len(names)):
        for k in range(i + 1, len(names)):
            v1 = names[i]
            v2 = names[k]
            
            if not ex_network.has_edge(v1, v2):
                ex_network.add_edge(v1, v2, weight = 0)
            
            ex_network[v1][v2]["weight"] += 1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [10]:
#Compute centralities
#Degree, Weighted Degree, Closeness, Betweenness
#Make a large dataframe with all the centralities

exDegDict = dict(ex_network.degree())
exDegDF = pd.DataFrame.from_dict(exDegDict, orient = "index", columns = ["DEG"])

exWDegDict = dict(ex_network.degree(weight="weight"))
exWDegDF = pd.DataFrame.from_dict(exWDegDict, orient = "index", columns = ["WDEG"])

exClsDict = dict(nx.closeness_centrality(ex_network))
exClsDF = pd.DataFrame.from_dict(exClsDict, orient = "index", columns = ["CLS"])

exBtwDict = dict(nx.betweenness_centrality(ex_network))
exBtwDF = pd.DataFrame.from_dict(exBtwDict, orient = "index", columns = ["BTW"])

exAllDF = pd.DataFrame()
exAllDF = exAllDF.merge(exDegDF, how = "outer", left_index=True, right_index=True)
exAllDF = exAllDF.merge(exClsDF, how = "outer", left_index=True, right_index=True)
exAllDF = exAllDF.merge(exBtwDF, how = "outer", left_index=True, right_index=True)
exAllDF = exAllDF.merge(exWDegDF, how = "outer", left_index=True, right_index=True)

exAllDF

,DEG,CLS,BTW,WDEG
навальный,107,0.350685,4.773421e-03,212
чубайс,4,0.246599,0.000000e+00,4
колядин,4,0.246599,0.000000e+00,4
квачков,4,0.246599,0.000000e+00,4
удальцов,103,0.349340,1.795382e-03,204
валеев,106,0.349743,5.163513e-03,206
калачев,4,0.250944,0.000000e+00,4
кабанов,4,0.250944,0.000000e+00,4
юсуфов,13,0.313637,1.175844e-03,13
выборный,4,0.250944,0.000000e+00,4


In [11]:
exAllDF.to_excel("exploratiry_cent.xlsx")

In [12]:
confDict = open("dict.txt", encoding="utf8").read().split()
confDict = set(confDict)
confDict

{'адагамов',
 'баронова',
 'верзилов',
 'волков',
 'володин',
 'воробьев',
 'гитлер',
 'иванов',
 'калашников',
 'красовский',
 'ленин',
 'лимонов',
 'литвинов',
 'медведев',
 'митрохин',
 'навальный',
 'немцов',
 'пархоменко',
 'порошенко',
 'просвирнин',
 'путин',
 'собчак',
 'сталин',
 'удальцов',
 'улицкая',
 'ходорковский'}

In [13]:
#Note that this works MUCH MUCH better with mystem
tokenizer = nltk.tokenize.RegexpTokenizer("[а-яА-Я]+")
tokenizedTexts = [tokenizer.tokenize(text) for text in texts]
morphA = pymorphy2.MorphAnalyzer()
normalizedTexts = []
for tokens in tokenizedTexts:
    normalizedText = [morphA.parse(token)[0].normal_form for token in tokens]
    normalizedTexts.append(normalizedText)
"|".join(normalizedTexts[0])

'страница|яков|миркина|надо|наращивать|личный|потребление|чтобы|весь|долгий|жить|учитель|наконец|избавить|от|составление|бесконечный|отчёт|новый|законопроект|снизить|бумажный|нагрузка|на|педагог|страница|событие|и|комментарий|событие|и|комментарий|российский|газета|февраль|страница|российский|газета|февраль|вторник|событие|и|комментарий|версия|документ|доступный|в|система|для|просмотр|авторизоваться|в|система|и|перейти|по|ссылка'

In [14]:
conf_network = nx.Graph()
for text in normalizedTexts:
    text = set(text)
    names = text & confDict
    names = list(names) #We want an ordered collection here
    for i in range(0, len(names)):
        for k in range(i + 1, len(names)):
            v1 = names[i]
            v2 = names[k]
            
            if not conf_network.has_edge(v1, v2):
                conf_network.add_edge(v1, v2, weight = 0)
            
            conf_network[v1][v2]["weight"] += 1

In [15]:
def makeTranslitNet(graph):
    origNodes = list(graph.nodes)
    translitNodes = [transliterate.translit(word, language_code='ru', reversed=True) for word in origNodes]
    translitNodes = [re.sub(r'\W+', '', word) for word in translitNodes]
    nodeNamesMap = dict(zip(origNodes, translitNodes))
    translitGraph = nx.relabel_nodes(graph, nodeNamesMap)
    return translitGraph


In [16]:
nx.write_pajek(makeTranslitNet(conf_network), "confNet.net")